In [95]:
import re, orodja
import pandas as pd
file = 'osu_index.txt'
aux_file = 'data-auxf.txt'
page = 'https://osu.ppy.sh/beatmapsets?sort=difficulty_desc&s=ranked'

V tega bomo dal regex-e

In [96]:
curr_re = r'\{"beatmapsets":\[(.*?)\]\}\]'
data_block_re = r'\{.*?\}\]\}'
data_re = re.compile(
    r'{"artist":(?P<artist>.*?),.*?,'
    r'"creator":(?P<creator>.*),'
    r'"favourite_count":(?P<fav_count>\d*),.*?,'
    r'"id":(?P<id>\d*),.*?,'
    r'"play_count":(?P<play_count>\d*),.*?,'
    r'"status":(?P<status>.*),'
    r'"title":(?P<title>.*?),.*?,'
    r'"bpm":(?P<bpm>\d*),.*?}]}',
    flags=re.DOTALL
)

In [97]:
artist_re = r''
creator_re = r''
favourite_count = r''
id_re = r''
play_count_re = r''
status_re = r''
title_re = r''
bpm_re = r''

# Tukej delamo posamezne odseke kode

In [98]:
def scrape(fin: str, out: str) -> None:
    regex = re.compile(curr_re, flags=re.DOTALL) #re.DOTALL postav piko na vse, vkljucno s novo vrstico
    with open(fin) as f:
        stuff = f.read()
        with open(out, 'w', encoding='UTF8')as o:
            print(re.findall(regex, stuff), file=o)

In [117]:
def make_blocks(filename: str) -> list:
    l = []
    with open(filename) as f:
        stuff= f.read()
    regex = re.compile(data_block_re, flags=re.DOTALL)
    for block in re.findall(regex, stuff):
        l.append(block)
    return l

In [118]:
def scrape_block(block: str) -> dict:
    if not block: return {}
    map = data_re.search(block).groupdict()
    map['fav_count'] = int(map['fav_count'])
    map['id'] = int(map['id'])
    map['play_count'] = int(map['play_count'])
    map['bpm'] = int(map['bpm']) 
    return map

In [119]:
def insert_json(data_in: list, out: str) -> None:
    return orodja.zapisi_json(data_in, out)
    #slight hack he he

Ta del nadomešča (if \_\_name__ == '\_\_main__')

In [120]:
orodja.shrani_spletno_stran(page, file, vsili_prenos=True)
scrape(file, aux_file)
data_blocks = make_blocks(aux_file)
data = []
for block in data_blocks:
    data.append(scrape_block(block))
insert_json(data, 'data.json')

AttributeError: 'NoneType' object has no attribute 'groupdict'

### Random stuff

In [121]:
print(data_blocks, len(data_blocks))

['{"artist":"sana","artist_unicode":"\\\\u9396\\\\u90a3","covers":{"cover":"https:\\\\/\\\\/assets.ppy.sh\\\\/beatmaps\\\\/1839405\\\\/covers\\\\/cover.jpg?1666707336","cover@2x":"https:\\\\/\\\\/assets.ppy.sh\\\\/beatmaps\\\\/1839405\\\\/covers\\\\/cover@2x.jpg?1666707336","card":"https:\\\\/\\\\/assets.ppy.sh\\\\/beatmaps\\\\/1839405\\\\/covers\\\\/card.jpg?1666707336","card@2x":"https:\\\\/\\\\/assets.ppy.sh\\\\/beatmaps\\\\/1839405\\\\/covers\\\\/card@2x.jpg?1666707336","list":"https:\\\\/\\\\/assets.ppy.sh\\\\/beatmaps\\\\/1839405\\\\/covers\\\\/list.jpg?1666707336","list@2x":"https:\\\\/\\\\/assets.ppy.sh\\\\/beatmaps\\\\/1839405\\\\/covers\\\\/list@2x.jpg?1666707336","slimcover":"https:\\\\/\\\\/assets.ppy.sh\\\\/beatmaps\\\\/1839405\\\\/covers\\\\/slimcover.jpg?1666707336","slimcover@2x":"https:\\\\/\\\\/assets.ppy.sh\\\\/beatmaps\\\\/1839405\\\\/covers\\\\/slimcover@2x.jpg?1666707336"},"creator":"Seiran-","favourite_count":10,"hype":null,"id":1839405,"nsfw":false,"offset":0,"p

In [122]:
print(data_blocks[0],data_blocks[1])

{"artist":"sana","artist_unicode":"\\u9396\\u90a3","covers":{"cover":"https:\\/\\/assets.ppy.sh\\/beatmaps\\/1839405\\/covers\\/cover.jpg?1666707336","cover@2x":"https:\\/\\/assets.ppy.sh\\/beatmaps\\/1839405\\/covers\\/cover@2x.jpg?1666707336","card":"https:\\/\\/assets.ppy.sh\\/beatmaps\\/1839405\\/covers\\/card.jpg?1666707336","card@2x":"https:\\/\\/assets.ppy.sh\\/beatmaps\\/1839405\\/covers\\/card@2x.jpg?1666707336","list":"https:\\/\\/assets.ppy.sh\\/beatmaps\\/1839405\\/covers\\/list.jpg?1666707336","list@2x":"https:\\/\\/assets.ppy.sh\\/beatmaps\\/1839405\\/covers\\/list@2x.jpg?1666707336","slimcover":"https:\\/\\/assets.ppy.sh\\/beatmaps\\/1839405\\/covers\\/slimcover.jpg?1666707336","slimcover@2x":"https:\\/\\/assets.ppy.sh\\/beatmaps\\/1839405\\/covers\\/slimcover@2x.jpg?1666707336"},"creator":"Seiran-","favourite_count":10,"hype":null,"id":1839405,"nsfw":false,"offset":0,"play_count":28,"preview_url":"\\/\\/b.ppy.sh\\/preview\\/1839405.mp3","source":"","spotlight":false,"st

In [126]:
for i in range(50):
    print(scrape_block(data_blocks[i]))

{'artist': '"sana"', 'creator': '"Seiran-"', 'fav_count': 10, 'id': 1839405, 'play_count': 28, 'status': '"ranked"', 'title': '"Sunset March"', 'bpm': 130}
{'artist': '"sana"', 'creator': '"DenYi"', 'fav_count': 10, 'id': 1794544, 'play_count': 38, 'status': '"ranked"', 'title': '"Sunset March"', 'bpm': 130}
{'artist': '"Imperial Circus Dead Decadence"', 'creator': '"Hantrik"', 'fav_count': 73, 'id': 1776682, 'play_count': 128, 'status': '"ranked"', 'title': '"Hyakki Yakou -Pandemonic Night Parade-"', 'bpm': 200}
{'artist': '"ReeK"', 'creator': '"-mint-"', 'fav_count': 34, 'id': 1836713, 'play_count': 129, 'status': '"ranked"', 'title': '"Multigenre Smackdown"', 'bpm': 160}
{'artist': '"Ohara Yuiko"', 'creator': '"bigh"', 'fav_count': 2, 'id': 1669080, 'play_count': 229, 'status': '"ranked"', 'title': '"Tabibito no Uta (TV Size)"', 'bpm': 137}
{'artist': '"Kardashev"', 'creator': '"kiwies"', 'fav_count': 150, 'id': 1770210, 'play_count': 520, 'status': '"ranked"', 'title': '"Snow-Sleep

AttributeError: 'NoneType' object has no attribute 'groupdict'